## 1. Importacion de librerias

In [82]:
# Instalación de librerias
# librería Natural Language Toolkit, usada para trabajar con textos 
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import pandas as pd
import numpy as np
import sys
from pandas_profiling import ProfileReport

import re, string, unicodedata
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.feature_extraction.text import TfidfVectorizer

import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\leane\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\leane\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\leane\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## 2. Carga de datos

In [162]:
scopusData = pd.read_csv('data/scopusArticles.csv', sep=',', encoding = 'utf-8')
scopusData.head()

,Title,Link,Abstract,Author Keywords,Index Keywords
0,Constructive Alignment by Portfolio Exams for ...,https://www.scopus.com/inward/record.uri?eid=2...,The paper shows how the useful paradigm of con...,competency graphs; Constructive alignment; exa...,Advanced control; Competency graph; Constructi...
1,Realising the promises of artificial intellige...,https://www.scopus.com/inward/record.uri?eid=2...,To support manufacturing firms in realising th...,Artificial intelligence; CRISP-DM; machine lea...,Cost engineering; Economic and social effects;...
2,Application of Geographic Information Systems ...,https://www.scopus.com/inward/record.uri?eid=2...,New technologies and global datasets enable tr...,data and data science; economic impacts; geogr...,COVID-19; Data visualization; Economic analysi...
3,Optimization algorithms and investment portfol...,https://www.scopus.com/inward/record.uri?eid=2...,Purpose: This paper aims to examine from commo...,Analytics; Artificial intelligence; Business a...,NaN
4,Harnessing the power of synthetic data in heal...,https://www.scopus.com/inward/record.uri?eid=2...,Data-driven decision-making in modern healthca...,NaN,Data privacy; Decision making; Financial data ...


In [163]:
originalFrameworkData = pd.read_csv('data/verifiedArticles.csv', sep=',', encoding = 'ANSI')
originalFrameworkData.head()

,Title,Authors,Year,Cited by,Abstract,DOI,Link,EID,Criterios,Publicación,Cómo aporta la analítica?,PUBLISHER,H-Index,Quartiles,Nota,URL,Unnamed: 16
0,A Dashboard to Support Management of Business ...,Santiago Rivera D.; Shanks G.,2015,22.0,Business analytics (BA) systems create value a...,10.1080/12460125.2015.994335,https://www.scopus.com/inward/record.uri?eid=2...,2-s2.0-84924258742,Alineación con la estrategia y objetivos,Journal of Decision Systems,Cómo alinearse con la estrategía a partir de l...,NaN,28.0,Library and Information Sciences \t2022 \tQ1\n...,NaN,NaN,researchgate
1,Project portfolio selection problems: A review...,Mohagheghi V.; Mousavi S.M.; Antuchevi?ien? J....,2019,18.0,Project portfolio selection has been the focus...,10.3846/tede.2019.11410,https://www.scopus.com/inward/record.uri?eid=2...,2-s2.0-85076850409,Criterios para establecer objetivos de valor,Technological and Economic Development of Economy,Cómo la analítica aporta en la selección de pr...,Vilnius Gediminas Technical University,59.0,Finance\t 2022 \tQ2,NaN,NaN,NaN
2,A decision-making framework for project portfo...,Sampath S.; Gel E.S.; Fowler J.W.; Kempf K.G.,2015,12.0,The work we describe addresses the problem of ...,10.1287/inte.2015.0809,https://www.scopus.com/inward/record.uri?eid=2...,2-s2.0-84949207726,Alineación con la estrategia y objetivos,Interfaces,Cómo la analítica aporta en la gestión y plane...,NaN,69.0,Management of Technology and Innovation \t2022...,NaN,NaN,NaN
3,Big data analytics in innovation processes: wh...,Capurro R.; Fiorentino R.; Garzella S.; Giudic...,2021,7.0,Purpose: The purpose of this paper is to analy...,10.1108/EJIM-05-2021-0256,https://www.scopus.com/inward/record.uri?eid=2...,2-s2.0-85110841823,Optimización,European Journal of Innovation\nManagement,Cómo la analítica permite optimizar la gestión...,Emerald Group Publishing Ltd.,74.0,Management of Technology and Innovation \t2022...,NaN,NaN,NaN
4,A Data-driven project categorization process f...,"El bok G., Berrado A.",2022,2.0,Purpose: Categorizing projects allows for bett...,10.1108/JM2-10-2020-0257,https://www.scopus.com/inward/record.uri?eid=2...,2-s2.0-85109023786,Selección,Journal of Modelling in\nManagement,NaN,NaN,37.0,Decision Sciences (miscellaneous) \t2022 \tQ2\...,NaN,NaN,NaN


## 3. Entendimiento de datos

In [164]:
print("Number of initial documents in Scopus dataframe: " + str(scopusData.shape[0]))

Number of initial documents in Scopus dataframe: 2368


In [165]:
print("Number of duplicated documents in Scopus dataframe: " + str(scopusData.duplicated().sum()))

Number of duplicated documents in Scopus dataframe: 407


In [166]:
scopusData = scopusData.drop_duplicates()
print("Number of different documents in Scopus dataframe: " + str(scopusData.shape[0]))

Number of different documents in Scopus dataframe: 1961


In [167]:
scopusData.isnull().sum(axis = 0)

Title                 0
Link                  0
Abstract              0
Author Keywords     550
Index Keywords     1281
dtype: int64

In [168]:
print("Number of initial documents on the framework file: " + str(originalFrameworkData.shape[0]))

Number of initial documents on the framework file: 69


In [169]:
originalFrameworkData.isnull().sum(axis = 0)

Title                         0
Authors                       0
Year                          0
Cited by                     13
Abstract                      0
DOI                          16
Link                          1
EID                           1
Criterios                    24
Publicación                  29
Cómo aporta la analítica?    48
PUBLISHER                    38
H-Index                      38
Quartiles                    39
Nota                         61
URL                          62
Unnamed: 16                  68
dtype: int64

In [170]:
originalFrameworkData = originalFrameworkData[originalFrameworkData["Criterios"].notna()]
print("Number of significative documents on the framework file: " + str(originalFrameworkData.shape[0]))

Number of significative documents on the framework file: 45


In [171]:
originalFrameworkData["Criterios"].unique()

array(['Alineación con la estrategia y objetivos',
       'Criterios para establecer objetivos de valor', 'Optimización',
       'Selección', 'Criterios para el monitoreo y medición de valor.',
       'Medición o cuantificación de riesgos', 'Gobierno del portafolio',
       'Criterios financieros para la selección de proyectos',
       'Identificación de riesgos', 'Gestión de capacidades',
       'Métricas para el monitoreo del portafolio de proyectos',
       'Criterios para la medición de beneficios.',
       'Gestión de intersados', 'Papel de la analitica?', 'KPI',
       'Criterios financieros para la ejecución del portfolio ',
       'Priorización'], dtype=object)

## 4. Preparación de datos

### 4.1. Organización de datos iniciales

In [172]:
modifiedScopusData = scopusData.copy()
modifiedScopusData.head()

,Title,Link,Abstract,Author Keywords,Index Keywords
0,Constructive Alignment by Portfolio Exams for ...,https://www.scopus.com/inward/record.uri?eid=2...,The paper shows how the useful paradigm of con...,competency graphs; Constructive alignment; exa...,Advanced control; Competency graph; Constructi...
1,Realising the promises of artificial intellige...,https://www.scopus.com/inward/record.uri?eid=2...,To support manufacturing firms in realising th...,Artificial intelligence; CRISP-DM; machine lea...,Cost engineering; Economic and social effects;...
2,Application of Geographic Information Systems ...,https://www.scopus.com/inward/record.uri?eid=2...,New technologies and global datasets enable tr...,data and data science; economic impacts; geogr...,COVID-19; Data visualization; Economic analysi...
3,Optimization algorithms and investment portfol...,https://www.scopus.com/inward/record.uri?eid=2...,Purpose: This paper aims to examine from commo...,Analytics; Artificial intelligence; Business a...,NaN
4,Harnessing the power of synthetic data in heal...,https://www.scopus.com/inward/record.uri?eid=2...,Data-driven decision-making in modern healthca...,NaN,Data privacy; Decision making; Financial data ...


In [173]:
modifiedScopusData['Author Keywords'] = modifiedScopusData['Author Keywords'].fillna("")
modifiedScopusData['Index Keywords'] = modifiedScopusData['Index Keywords'].fillna("")
modifiedScopusData['Abstract'] = modifiedScopusData['Abstract'] + " " + modifiedScopusData['Title'] + " " + modifiedScopusData['Author Keywords'] + " " +  modifiedScopusData['Index Keywords']
modifiedScopusData.head()

,Title,Link,Abstract,Author Keywords,Index Keywords
0,Constructive Alignment by Portfolio Exams for ...,https://www.scopus.com/inward/record.uri?eid=2...,The paper shows how the useful paradigm of con...,competency graphs; Constructive alignment; exa...,Advanced control; Competency graph; Constructi...
1,Realising the promises of artificial intellige...,https://www.scopus.com/inward/record.uri?eid=2...,To support manufacturing firms in realising th...,Artificial intelligence; CRISP-DM; machine lea...,Cost engineering; Economic and social effects;...
2,Application of Geographic Information Systems ...,https://www.scopus.com/inward/record.uri?eid=2...,New technologies and global datasets enable tr...,data and data science; economic impacts; geogr...,COVID-19; Data visualization; Economic analysi...
3,Optimization algorithms and investment portfol...,https://www.scopus.com/inward/record.uri?eid=2...,Purpose: This paper aims to examine from commo...,Analytics; Artificial intelligence; Business a...,
4,Harnessing the power of synthetic data in heal...,https://www.scopus.com/inward/record.uri?eid=2...,Data-driven decision-making in modern healthca...,,Data privacy; Decision making; Financial data ...


In [174]:
modifiedScopusData = modifiedScopusData.drop(columns = ['Link', 'Index Keywords', 'Author Keywords'])
modifiedScopusData.head()

,Title,Abstract
0,Constructive Alignment by Portfolio Exams for ...,The paper shows how the useful paradigm of con...
1,Realising the promises of artificial intellige...,To support manufacturing firms in realising th...
2,Application of Geographic Information Systems ...,New technologies and global datasets enable tr...
3,Optimization algorithms and investment portfol...,Purpose: This paper aims to examine from commo...
4,Harnessing the power of synthetic data in heal...,Data-driven decision-making in modern healthca...


In [175]:
modifiedFrameworkData = originalFrameworkData.copy()
modifiedFrameworkData = modifiedFrameworkData[["Title", "Abstract", "Criterios"]]
modifiedFrameworkData.head()

,Title,Abstract,Criterios
0,A Dashboard to Support Management of Business ...,Business analytics (BA) systems create value a...,Alineación con la estrategia y objetivos
1,Project portfolio selection problems: A review...,Project portfolio selection has been the focus...,Criterios para establecer objetivos de valor
2,A decision-making framework for project portfo...,The work we describe addresses the problem of ...,Alineación con la estrategia y objetivos
3,Big data analytics in innovation processes: wh...,Purpose: The purpose of this paper is to analy...,Optimización
4,A Data-driven project categorization process f...,Purpose: Categorizing projects allows for bett...,Selección


In [176]:
modifiedFrameworkData['Abstract'] = modifiedFrameworkData['Abstract'] + " " + modifiedFrameworkData['Title']
modifiedFrameworkData.head()

,Title,Abstract,Criterios
0,A Dashboard to Support Management of Business ...,Business analytics (BA) systems create value a...,Alineación con la estrategia y objetivos
1,Project portfolio selection problems: A review...,Project portfolio selection has been the focus...,Criterios para establecer objetivos de valor
2,A decision-making framework for project portfo...,The work we describe addresses the problem of ...,Alineación con la estrategia y objetivos
3,Big data analytics in innovation processes: wh...,Purpose: The purpose of this paper is to analy...,Optimización
4,A Data-driven project categorization process f...,Purpose: Categorizing projects allows for bett...,Selección


### 4.2. Limpieza de datos

In [177]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stop_words:
            new_words.append(word)
    return new_words
        
    
def preprocessing(words):
    words = to_lowercase(words)
    words = replace_numbers(words)
    words = remove_punctuation(words)
    words = remove_non_ascii(words)
    words = remove_stopwords(words)
    return words

### 4.3. Tokenización

In [178]:
tokenizedFrameworkData = modifiedFrameworkData.copy()
tokenizedFrameworkData.head()

,Title,Abstract,Criterios
0,A Dashboard to Support Management of Business ...,Business analytics (BA) systems create value a...,Alineación con la estrategia y objetivos
1,Project portfolio selection problems: A review...,Project portfolio selection has been the focus...,Criterios para establecer objetivos de valor
2,A decision-making framework for project portfo...,The work we describe addresses the problem of ...,Alineación con la estrategia y objetivos
3,Big data analytics in innovation processes: wh...,Purpose: The purpose of this paper is to analy...,Optimización
4,A Data-driven project categorization process f...,Purpose: Categorizing projects allows for bett...,Selección


In [179]:
stop_words = stopwords.words('english')

In [180]:
tokenizedFrameworkData["Abstract"] = tokenizedFrameworkData["Abstract"].apply(contractions.fix)
tokenizedFrameworkData['Words'] = tokenizedFrameworkData['Abstract'].apply(word_tokenize).apply(preprocessing)
tokenizedFrameworkData.head()

,Title,Abstract,Criterios,Words
0,A Dashboard to Support Management of Business ...,Business analytics (BA) systems create value a...,Alineación con la estrategia y objetivos,"[business, analytics, ba, systems, create, val..."
1,Project portfolio selection problems: A review...,Project portfolio selection has been the focus...,Criterios para establecer objetivos de valor,"[project, portfolio, selection, focus, many, s..."
2,A decision-making framework for project portfo...,The work we describe addresses the problem of ...,Alineación con la estrategia y objetivos,"[work, describe, addresses, problem, deciding,..."
3,Big data analytics in innovation processes: wh...,Purpose: The purpose of this paper is to analy...,Optimización,"[purpose, purpose, paper, analyze, dynamic, ca..."
4,A Data-driven project categorization process f...,Purpose: Categorizing projects allows for bett...,Selección,"[purpose, categorizing, projects, allows, bett..."


### 4.4. Normalización

In [181]:
stemmer = LancasterStemmer()
lemmatizer = WordNetLemmatizer()

In [182]:
def stem_words(words):
    """Stem words in list of tokenized words"""
    new_words = []
    for word in words:
        new_word = stemmer.stem(word)
        new_words.append(new_word)
    return new_words

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    new_words = []
    for word in words:
        new_word = lemmatizer.lemmatize(word)
        new_words.append(new_word)
    return new_words

def stem_and_lemmatize(words):
    stems = stem_words(words)
    lemmas = lemmatize_verbs(words)
    return stems + lemmas

In [183]:
tokenizedFrameworkData['Words'] = tokenizedFrameworkData['Words'].apply(stem_and_lemmatize)
tokenizedFrameworkData.head()

,Title,Abstract,Criterios,Words
0,A Dashboard to Support Management of Business ...,Business analytics (BA) systems create value a...,Alineación con la estrategia y objetivos,"[busy, analys, ba, system, cre, valu, provid, ..."
1,Project portfolio selection problems: A review...,Project portfolio selection has been the focus...,Criterios para establecer objetivos de valor,"[project, portfolio, select, foc, many, schola..."
2,A decision-making framework for project portfo...,The work we describe addresses the problem of ...,Alineación con la estrategia y objetivos,"[work, describ, address, problem, decid, proje..."
3,Big data analytics in innovation processes: wh...,Purpose: The purpose of this paper is to analy...,Optimización,"[purpos, purpos, pap, analys, dynam, cap, pers..."
4,A Data-driven project categorization process f...,Purpose: Categorizing projects allows for bett...,Selección,"[purpos, categ, project, allow, bet, align, po..."


In [184]:
tokenizedFrameworkData['Words'] = tokenizedFrameworkData['Words'].apply(lambda x: ' '.join(map(str, x)))
tokenizedFrameworkData.head()

,Title,Abstract,Criterios,Words
0,A Dashboard to Support Management of Business ...,Business analytics (BA) systems create value a...,Alineación con la estrategia y objetivos,busy analys ba system cre valu provid competit...
1,Project portfolio selection problems: A review...,Project portfolio selection has been the focus...,Criterios para establecer objetivos de valor,project portfolio select foc many scholars las...
2,A decision-making framework for project portfo...,The work we describe addresses the problem of ...,Alineación con la estrategia y objetivos,work describ address problem decid projectfund...
3,Big data analytics in innovation processes: wh...,Purpose: The purpose of this paper is to analy...,Optimización,purpos purpos pap analys dynam cap perspect ro...
4,A Data-driven project categorization process f...,Purpose: Categorizing projects allows for bett...,Selección,purpos categ project allow bet align portfolio...


In [185]:
tokenizedFrameworkData["Criteria Code"] = pd.factorize(tokenizedFrameworkData['Criterios'])[0]
tokenizedFrameworkData.head()

,Title,Abstract,Criterios,Words,Criteria Code
0,A Dashboard to Support Management of Business ...,Business analytics (BA) systems create value a...,Alineación con la estrategia y objetivos,busy analys ba system cre valu provid competit...,0
1,Project portfolio selection problems: A review...,Project portfolio selection has been the focus...,Criterios para establecer objetivos de valor,project portfolio select foc many scholars las...,1
2,A decision-making framework for project portfo...,The work we describe addresses the problem of ...,Alineación con la estrategia y objetivos,work describ address problem decid projectfund...,0
3,Big data analytics in innovation processes: wh...,Purpose: The purpose of this paper is to analy...,Optimización,purpos purpos pap analys dynam cap perspect ro...,2
4,A Data-driven project categorization process f...,Purpose: Categorizing projects allows for bett...,Selección,purpos categ project allow bet align portfolio...,3


In [186]:
vectorizer = TfidfVectorizer()
sparseMatrix = vectorizer.fit_transform(tokenizedFrameworkData['Abstract'])
sparseMatrix.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.05897435, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.09710986,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [187]:
vectorizer.get_feature_names_out()

array(['01', '131', '138', ..., 'yet', 'yield', 'zone'], dtype=object)